In [1]:
from transformers import AutoModel, AutoTokenizer
from string import punctuation
import torch
import pandas as pd
import pickle
import json
import spacy
import gc

In [5]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [7]:
data = pd.read_excel('../datasets/Датасет иконок.xlsx',).iloc[1:, 1:]
data.columns = ['ICON_NAME', 'ICON_DESCRIPTION']
data.index = data['ICON_NAME']
data = data['ICON_DESCRIPTION'].map(lambda a: a.lower())
print(data.shape)
data.head()

(288,)


ICON_NAME
login.svg                      вход, авторизация, начало работы
arrow-up-square.svg              рост, движение вверх, прогресс
arrow-left-tail.svg                    возврат назад, шаг назад
arrows-exchange-2.svg       обмен, взаимодействие, переключение
arrow-back-up.svg        откат, возврат к предыдущему состоянию
Name: ICON_DESCRIPTION, dtype: object

In [8]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [9]:
encoded_input = tokenizer(data.to_list(), padding=True, return_tensors='pt')
with torch.no_grad():
    model_output = vectorizer(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [7]:
torch.save(sentence_embeddings, '../datasets/embedded_icons.pt')

In [14]:
indexated_icons

{0: 'login.svg',
 1: 'arrow-up-square.svg',
 2: 'arrow-left-tail.svg',
 3: 'arrows-exchange-2.svg',
 4: 'arrow-back-up.svg',
 5: 'corner-left-up.svg',
 6: 'circle-arrow-left.svg',
 7: 'circle-chevron-up.svg',
 8: 'square-rounded-chevrons-up.svg',
 9: 'caret-down.svg',
 10: 'transition-right.svg',
 11: 's-turn-down.svg',
 12: 'corner-up-left-double.svg',
 13: 'arrows-move-horizontal.svg',
 14: 'arrow-up-tail.svg',
 15: 'arrows-diff.svg',
 16: 'trending-down.svg',
 17: 'arrow-big-right-line.svg',
 18: 'square-chevrons-up.svg',
 19: 'chevron-right.svg',
 20: 'arrow-down-right.svg',
 21: 'switch.svg',
 22: 'circle-arrow-down-right.svg',
 23: 'chevrons-down-right.svg',
 24: 'circle-arrow-down-left.svg',
 25: 'arrow-left-rhombus.svg',
 26: 'arrow-right-tail.svg',
 27: 'arrow-loop-left.svg',
 28: 'square-chevron-left.svg',
 29: 'square-rounded-arrow-down.svg',
 30: 'corner-left-down.svg',
 31: 'corner-down-right-double.svg',
 32: 'arrow-bar-down.svg',
 33: 'arrows-down-up.svg',
 34: 'caret-ri

In [13]:
indexated_icons = dict(enumerate(data.index))
with open('../datasets/indexated_icons.json', 'w') as f:
    json.dump(indexated_icons, f)

# Test on real data

In [53]:
text_example_1 = "Москвичи, зарегистрировавшиеся до 8 сентября на розыгрыш дополнительных призов, "+\
    "смогут претендовать на 15 тысяч баллов в одной из категорий: «Путешествия и культура», "+\
    "«Техника для дома» или «Спорт»."

text_example_2 = "В электронном "+\
    "голосовании на выборах депутатов Московской городской Думы, муниципальных депутатов ТиНАО и района "+\
    "Куркино приняли участие уже 450 тысяч человек. Отдать свой голос можно до 20:00 8 сентября. "+\
    "В настоящее время система работает штатно, никаких сбоев или задержек в ее работе не зафиксировано."

text_exaple_3 = "Специалисты отреставрировали кирпичную кладку, воссоздали "+\
    "декоративные элементы, привели в порядок мерлоны (зубцы), карнизы, своды и кровлю из "+\
    "керамического лемеха. В нише над проездной аркой установили художественное панно. Сейчас работы "+\
    "завершены, и горожане могут видеть преобразившийся памятник архитектуры. Список победителей и "+\
    "информацию о том, как потратить городские баллы, можно найти на сайте «Миллион призов»."

In [1]:
import torch

In [17]:
icons_embeddings = torch.load('../datasets/embedded_icons.pt')

/tmp/ipykernel_783529/962715775.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  icons_embeddings = torch.load('../datasets/embedded_icons.pt')


In [62]:
n = -1

In [63]:
encoded_input = tokenizer([text_example_1], padding=True, return_tensors='pt')
with torch.no_grad():
    model_output = vectorizer(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).repeat(icons_embeddings.shape[0], 1)
print(sentence_embeddings.shape, icons_embeddings.shape)
res = torch.nn.functional.cosine_similarity(sentence_embeddings, icons_embeddings)
print(res[res.argsort()[n]])
print(indexated_icons[res.argsort()[n].item()])

torch.Size([288, 1024]) torch.Size([288, 1024])
tensor(0.6315)
share-3.svg


In [64]:
print(text_example_2)
encoded_input = tokenizer([text_example_2], padding=True, return_tensors='pt')
with torch.no_grad():
    model_output = vectorizer(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).repeat(icons_embeddings.shape[0], 1)
print(sentence_embeddings.shape, icons_embeddings.shape)
res = torch.nn.functional.cosine_similarity(sentence_embeddings, icons_embeddings)
print(res[res.argsort()[n]])
print(indexated_icons[res.argsort()[n].item()])

В электронном голосовании на выборах депутатов Московской городской Думы, муниципальных депутатов ТиНАО и района Куркино приняли участие уже 450 тысяч человек. Отдать свой голос можно до 20:00 8 сентября. В настоящее время система работает штатно, никаких сбоев или задержек в ее работе не зафиксировано.
torch.Size([288, 1024]) torch.Size([288, 1024])
tensor(0.5243)
share-3.svg


In [65]:
print(text_exaple_3)
encoded_input = tokenizer([text_exaple_3], padding=True, return_tensors='pt')
with torch.no_grad():
    model_output = vectorizer(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).repeat(icons_embeddings.shape[0], 1)
print(sentence_embeddings.shape, icons_embeddings.shape)
res = torch.nn.functional.cosine_similarity(sentence_embeddings, icons_embeddings)
print(res[res.argsort()[n]])
print(indexated_icons[res.argsort()[n].item()])

Специалисты отреставрировали кирпичную кладку, воссоздали декоративные элементы, привели в порядок мерлоны (зубцы), карнизы, своды и кровлю из керамического лемеха. В нише над проездной аркой установили художественное панно. Сейчас работы завершены, и горожане могут видеть преобразившийся памятник архитектуры. Список победителей и информацию о том, как потратить городские баллы, можно найти на сайте «Миллион призов».
torch.Size([288, 1024]) torch.Size([288, 1024])
tensor(0.4857)
arrow-big-up-line.svg
